In [1]:
import tenpy
from tenpy.models.fermions_hubbard import FermionicHubbardChain
from tenpy.networks.mps import MPS
from tenpy.algorithms import dmrg
import time
import numpy as np
import json
from tools import solve_nonint
from scipy.optimize import minimize as minimize

/home/eoghanc/anaconda3/envs/ml-ldft-repo/lib/python3.8/site-packages/tenpy/tools/optimization.py:307: UserWarning: Couldn't load compiled cython code. Code will run a bit slower.
  warnings.warn("Couldn't load compiled cython code. Code will run a bit slower.")
/home/eoghanc/anaconda3/envs/ml-ldft-repo/lib/python3.8/site-packages/tenpy/models/fermions_hubbard.py:22: UserWarning: RESTRUCTURING
***********
* WARNING:
* The signs of hopping and chemical potential parameters were changed to the usual conventions!
* Moreover, "bose_hubbard.py" and "fermions_hubbard.py" models have now been consolidated into "hubbard.py".
***********
To avoid this warning, simply import the model class from `tenpy.models.hubbard` instead of `tenpy.models.fermions_hubbard`.
  warnings.warn(msg)
/home/eoghanc/anaconda3/envs/ml-ldft-repo/lib/python3.8/site-packages/tenpy/models/fermions_hubbard.py:23: FutureWarning: The module `tenpy.models.fermions_hubbard` is deprecated now.
  warnings.warn("The module `tenp

In [2]:
L=18
N=L//3
x = (L+1)/2

delta = np.sqrt(np.linspace(0,9,5)) # This distribution of the magnitude of disorder gives a good corresponding distribution of the densities
num = len(delta)

model_params = dict(cons_N='N', cons_Sz='Sz', t=1, U=4, mu=0, V=0, lattice="Chain", bc_MPS='finite',
                        order='default', L=L)
product_state = ["up", "down",0]*(L//3)
n_up = np.zeros((num, L))
n_dn = np.zeros((num, L))
v = np.zeros((num, L))
v_hxc = np.zeros((num, L))
norms = np.zeros(num)
d_err = np.zeros(num)
e = np.zeros(num)
emv = np.zeros(num)
e_ni = np.zeros(num)
e_hxc = np.zeros(num)
times = np.zeros(num)

In [3]:
########## DMRG ###########
for a, w in enumerate(delta):
    x = FermionicHubbardChain(model_params)
    x.manually_call_init_H = True
    V = w*np.random.random(L); V=V-np.mean(V)
    #V = efield(w, L)
    for b, y in enumerate(V):
        x.add_onsite_term(strength = y , i=b, op = 'Cdd Cd')
        x.add_onsite_term(strength = y , i=b, op = 'Cdu Cu')
    x.init_H_from_terms()
    psi = MPS.from_product_state(x.lat.mps_sites(), product_state, bc=x.lat.bc_MPS)
    dmrg_params = {
            'active_sites': 2,
            'mixer': False,
            'trunc_params': {'chi_max': 350},
            'N_sweeps_check': 10,
            'norm_tol':1E-6,
            'max_hours': 1}
    t1 = time.time()
    info = dmrg.run(psi, x, dmrg_params)
    times[a] = time.time() - t1
    n_dn[a] = psi.expectation_value('Cdd Cd', [i for i in range(L)])
    n_up[a] = psi.expectation_value('Cdu Cu', [i for i in range(L)])
    e[a] = info['E']
    emv[a] = e[a] - np.dot(n_up[a] + n_dn[a], V)
    v[a] = V
    norms[a] = info['sweep_statistics']['norm_err'][-1]
    print(str(a + 1)+'/'+str(num)+' DMRG calculations completed')
    print('DMRG time '+str(time.time() - t1))
    print('DMRG truncation error '+str(norms[a]))

1/5 DMRG calculations completed
DMRG time 113.53052544593811
DMRG truncation error 2.773537746156809e-09
2/5 DMRG calculations completed
DMRG time 116.32465982437134
DMRG truncation error 3.6937758752208224e-09
3/5 DMRG calculations completed
DMRG time 112.25773692131042
DMRG truncation error 1.7519115453542324e-09
4/5 DMRG calculations completed
DMRG time 117.3751871585846
DMRG truncation error 2.36727153355527e-09
5/5 DMRG calculations completed
DMRG time 110.49612236022949
DMRG truncation error 7.331621911229003e-10


In [4]:
######### Reverse engineering the density ###########
def cost_n(n,v, states):
    def cost(vxc):
        diag = solve_nonint(v+vxc, N, L)
        return np.sqrt(np.sum(np.square(diag[0] - n)))
    return cost

n_ = (n_up + n_dn)/2 # They should be roughly equal anyway... The difference between the two is a numerical artefact from the DMRG and is quite small

for j in range(num):
    V = v[j]
    n = n_[j]
    t1 = time.time()
    success = False
    temp_cost = cost_n(n, V, N)
    pred = minimize(temp_cost, np.zeros(L),method = 'BFGS', jac = '2-point', options = {'finite_diff_rel_step':1E-9})
    if pred.fun < 1E-6:
        success = True
    if success == True:
        result = solve_nonint(V + pred.x, N, L)
        e_ni[j] = result[1] - np.dot(V+pred.x, result[0])
        d_err[j] = pred.fun
        v_hxc[j] = pred.x - np.mean(pred.x)
        print(str(j + 1)+'/'+str(num)+' Reverse-engineering calculations completed')
        print('Reverse-engineering time', time.time() - t1)
        print('Reverse engineering error '+str(d_err[j]))

1/5 Reverse-engineering calculations completed
Reverse-engineering time 0.3222846984863281
Reverse engineering error 1.5022721427846488e-08
2/5 Reverse-engineering calculations completed
Reverse-engineering time 0.25910210609436035
Reverse engineering error 1.1677965778024764e-08
3/5 Reverse-engineering calculations completed
Reverse-engineering time 0.26981377601623535
Reverse engineering error 2.722773248604518e-09
4/5 Reverse-engineering calculations completed
Reverse-engineering time 0.24805903434753418
Reverse engineering error 4.7408260506099486e-08
5/5 Reverse-engineering calculations completed
Reverse-engineering time 0.30777931213378906
Reverse engineering error 2.3976425796006243e-09


In [7]:
e_hxc = emv - 2*e_ni

data = {'L': L, 
     'num' : num, 
     'n_up': n_up.tolist(), 
     'n_dn': n_dn.tolist(), 
     'e_mb': emv.tolist(), 
     'v': v.tolist(),
     'e_hxc': e_hxc.tolist(),
     'v_hxc': v_hxc.tolist(), 
     'norm_tol': norms.tolist(), 
     'reverse_engineering_error': d_err.tolist()}

# Uncomment below to save the data

#with open('data_json/exact_data/L'+str(L)+'_num'+str(num)+'_test.json', 'w') as f2:
#        json.dump(data, f2)
